In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
set_config(transform_output='pandas')

In [ ]:
songs = pd.read_csv("df_audio_features_5000_cleaned_whitespaces.csv")
songs.columns = songs.columns.str.strip()
songs = songs.drop("type", axis=1)
songs

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=22, n_init="auto", random_state=42)
kmeans.fit(songs.loc[:, ["danceability", "energy", "acousticness", "instrumentalness", "liveness", "valence"]])
kmeans.labels_
labeled_songs = songs.copy()
labeled_songs["cluster"] = kmeans.labels_
labeled_songs

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
scope = 'playlist-modify-public'
username = "Your user_name"
redirectUri = "http://localhost:7000/callback"
client_id = "Your Client_id"
client_secret = "Your_client_secret"

token = SpotifyOAuth(scope=scope,
                     username=username,
                     client_id=client_id,
                     client_secret=client_secret,
                     redirect_uri=redirectUri
                     )
spotifyObject = spotipy.Spotify(auth_manager = token)

In [ ]:
playlist_collection = {}
for i in range(15):
    playlist_name = f'my_playlist_cluster_{i}'
    playlist_description= 'this is a test list'
    playlist_id = spotifyObject.user_playlist_create(user=username,
                                                     name=playlist_name,
                                                     public=True,
                                                     description=playlist_description)['id']
    id_list = list(labeled_songs.loc[labeled_songs["cluster"] == i]
                                .sample(5)
                                .id)
    uris = [f'spotify:track:{str(id).strip()}' for id in id_list]
    spotifyObject.user_playlist_add_tracks(user=username,playlist_id=playlist_id,tracks=uris)
    playlist_collection[playlist_name] = playlist_id